In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Button, Layout, Output
from IPython.display import Image
from pathlib import Path
import numpy as np
from bqplot import OrdinalScale, LinearScale, Bars, Lines, Axis, Figure
from bqplot import pyplot as plt
import warnings

warnings.filterwarnings('ignore')


In [2]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
def create_expanded_output(description, button_style):
    return Output(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))


# Dice race!

## Roll either one or two dice and track their progress as they race up the chart. Which number will win?

* To roll the dice hit the "one dice" or "two dice" buttons.
* Add them up (easy for one dice) - and press the correct orange button. Try to get a thumbs up!
* Add the dice roll to the race by pressing one of the "add to race" buttons at the bottom of the grid. You can skip adding up the dice (pressing the orange buttons) if you like.
* You can clear the dice roll by pressing the "Clear Dice" button. This is optional.
* If playing on a phone it's better in landscape (-:

In [57]:
imgs = Path("images")
die = ['a']*6
die[0] = imgs/"one.png"
die[1] = imgs/"two.png"
die[2] = imgs/"three.png"
die[3] = imgs/"four.png"
die[4] = imgs/"five.png"
die[5] = imgs/"six.png"
thumbs_up = imgs/"thumbsup.png"
thumbs_down = imgs/"thumbsdown.png"

In [58]:
btn_one = Button(description='Roll one dice')
btn_two = Button(description='Roll two dice')
total = Output()
tracker_dict = {0: Output(),
               1: Output(),
               2: Output(),
               3: Output(),
               4: Output(),
               5: Output(),
               6: Output(),
               7: Output(),
               8: Output(),
               9: Output(),
               10: Output(),
               11: Output(),
               12: Output()}
for value in tracker_dict.values():
    value.value = 0

In [67]:
grid = GridspecLayout(9,10, width='1000px', height='450px')

grid[0,:6] = create_expanded_button('Clear Dice', 'primary')
grid[1:5,0] = create_expanded_button('One Dice', 'Success')
grid[1:5,1] = create_expanded_button('Two Dice', 'Success')
grid[1:5,2:4] = create_expanded_output('', 'info')
grid[1:5,4:6] = create_expanded_output('', 'info')
grid[7:,2:4] = create_expanded_output('result','')
grid[7:,:2] = create_expanded_button('add to race', 'primary')
grid[7:,4:6] = create_expanded_button('add to race', 'primary')
grid[:9,6:] = Output(layout=Layout(display='flex-shrink'))
for i in range(2):
    for j in range(6):
        grid[i+5,j] = create_expanded_button('{}'.format(i*6 + (j + 1)), 'warning')

grid

GridspecLayout(children=(Button(button_style='primary', description='Clear Dice', layout=Layout(grid_area='wid…

In [68]:
chart = Output(height = 600, width = 300)
chart

Output()

In [69]:
def on_click_throw_one(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    dice_throw = np.random.randint(6, size=None)
    with grid[1,2]: display(Image(die[dice_throw], width=195, height=195))
    # random ints are in (0, 5) representing dice throws in (1,6) - so add one!
    total.value = dice_throw+1

grid[1,0].on_click(on_click_throw_one)

In [70]:
def on_click_throw_two(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    dice_throw = np.random.randint(6, size=2)
    with grid[1,2]: display(Image(die[dice_throw[0]], width=195, height=195))
    with grid[1,4]: display(Image(die[dice_throw[1]], width=195, height=195))
    # random ints are in (0, 5) representing dice throws in (1,6) - so add two!
    total.value = dice_throw.sum()+2

grid[1,1].on_click(on_click_throw_two)

In [71]:
def on_click_clear(change):
    grid[1,2].clear_output()
    grid[1,4].clear_output()
    grid[7,3].clear_output()
    
grid[0,0].on_click(on_click_clear)

In [75]:
def on_click_add(change):
    tracker_dict[total.value].value += 1
    total.value = 0
    chart.clear_output()
    grid[0,8].clear_output()
    
    x_data = np.asarray(list(tracker_dict.keys())) + 1
    y_data = np.array(np.zeros(12),dtype=int)
    for key in tracker_dict.keys():
        if key >= 0: y_data[key - 1] = tracker_dict[key].value

    x_ord = OrdinalScale()
    y_sc = LinearScale()
    y_sc.min = 0
    y_sc.max = max(max(y_data),10)

    bar = Bars(x=x_data, y=y_data, scales={'x': x_ord, 'y':
    y_sc})

    ax_x = Axis(scale=x_ord, grid_lines='solid', label='Dice Roll')
    ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='d',
                tick_values= np.arange(0,max(max(y_data),10)+1), 
                grid_lines='solid', label='Number of rolls')

    race = Figure(marks=[bar], axes=[ax_x, ax_y], title='Race!!')
    
    with chart: display(race)
    with grid[0,8]: display(race,display='flex-shrink')
    
grid[7,0].on_click(on_click_add)
grid[7,5].on_click(on_click_add)

In [73]:
# There must be a more effecient way to do this! Like return the location of the button that has been pressed
# Anyway, twelve functions to check whether the correct button was pressed or not.
def on_click_one(change):
    grid[7,3].clear_output()
    if total.value == 1: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,0].on_click(on_click_one)

def on_click_two(change):
    grid[7,3].clear_output()
    if total.value == 2: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,1].on_click(on_click_two)

def on_click_three(change):
    grid[7,3].clear_output()
    if total.value == 3: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,2].on_click(on_click_three)

def on_click_four(change):
    grid[7,3].clear_output()
    if total.value == 4: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,3].on_click(on_click_four)

def on_click_five(change):
    grid[7,3].clear_output()
    if total.value == 5: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,4].on_click(on_click_five)

def on_click_six(change):
    grid[7,3].clear_output()
    if total.value == 6: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[5,5].on_click(on_click_six)

def on_click_seven(change):
    grid[7,3].clear_output()
    if total.value == 7: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,0].on_click(on_click_seven)

def on_click_eight(change):
    grid[7,3].clear_output()
    if total.value == 8: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,1].on_click(on_click_eight)

def on_click_nine(change):
    grid[7,3].clear_output()
    if total.value == 9: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,2].on_click(on_click_nine)

def on_click_ten(change):
    grid[7,3].clear_output()
    if total.value == 10: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,3].on_click(on_click_ten)

def on_click_eleven(change):
    grid[7,3].clear_output()
    if total.value == 11: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,4].on_click(on_click_eleven)

def on_click_twelve(change):
    grid[7,3].clear_output()
    if total.value == 12: 
        with grid[7,3]: display(Image(thumbs_up, width=195, height=100))
    else:
        with grid[7,3]: display(Image(thumbs_down, width=195, height=100))
grid[6,5].on_click(on_click_twelve)

In [54]:
####Experimenting with altair implementation of chart. It works, but not sure it improves it that much

# import pandas as pd
# index = range(1,13)
# data = [0] * 12
# dummy = data
# for i in range(1,13):
#     data[i-1] = tracker_dict[i].value

# data
# df=pd.DataFrame(np.zeros((12,2)),columns=['dice','quantity'])
# df.dice = range(1,13)
# df.quantity = data
# df

# mychart = alt.Chart(df).mark_bar().encode(
#     alt.X('dice:O'),
#     alt.Y('quantity:Q')
# )

In [17]:
# output = Output()
# with output: display(mychart)
# output